In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

from typing import Type

In [2]:
train_transform = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),

    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                         std=(0.2470, 0.2435, 0.2616))
])

In [3]:
train = datasets.CIFAR10("../../data", train=True, transform=train_transform,
                         download=True)
test = datasets.CIFAR10("../../data", train=False, transform=test_transform,
                        download=True)

In [4]:
train_dataloader = DataLoader(train, batch_size=64, shuffle=True, num_workers=12,
                              generator=torch.Generator().manual_seed(42))

test_dataloader = DataLoader(test, batch_size=64, shuffle=False, num_workers=12,
                             generator=torch.Generator().manual_seed(42))

In [ ]:
# ループを行うResident Blockを作る

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channel, out_channel, stride=1):
        super().__init__()

        # BatchNormを適用する際biasは無効化されるため最初からFalseにしておく

        # First Layer
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channel)

        # Second Layer
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channel)

        self.downsample = None

        # strideが1ではないせいでダウンサンプリングが生じる場合や
        # in_channelとout_channelが違うせいで特徴量マップの数が違う場合,
        # スキップ構造の元のxとConvやBatchNormで変換されたoutを
        # ブロードキャストできるように元のxを変換する
        if stride != 1 or in_channel != out_channel:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.batch_norm1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.batch_norm2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)
        out = out + identity

        out = F.relu(out)
        return out

In [6]:
class ResNet(nn.Module):
    def __init__(self, block: Type[BasicBlock], num_block: list, num_classes=10):
        super().__init__()

        self.in_channels = 64

        # 入力層
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)

        # 中間層(Layer 1~4)
        # 解像度を下げる代わりにchannel数を増やしてより表現力を高める
        self.layer1 = self._make_layer(block, 64, num_block[0], 1)
        self.layer2 = self._make_layer(block, 128, num_block[1], 2)
        self.layer3 = self._make_layer(block, 256, num_block[2], 2)
        self.layer4 = self._make_layer(block, 512, num_block[3], 2)

        self.fc = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block: Type[BasicBlock], out_channel, num_blocks, stride):
        stride = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in stride:
            layers.append(block(self.in_channels, out_channel, s))
            self.in_channels = out_channel * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.batch_norm1(out)
        out = F.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        # adaptive_avg_pool2dで解像度を1x1に変換する
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.fc(out)# adaptive_avg_pool2dによって解像度が1x1になったため、Linearの入力は512でいい
        return out

In [7]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

model = ResNet(BasicBlock, [3, 4, 6, 3]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

mps


In [12]:
def train_loop(model, dataloader, optimizer, criterion):
    model.train()

    for i, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()

        if i % 100 == 0 or i == 0:
            print(f"{i}/{len(dataloader)} Loss: {loss.item()}")

In [13]:
train_loop(model, train_dataloader, optimizer, criterion)

0/782 Loss: 1.2267749309539795
100/782 Loss: 1.5665042400360107
200/782 Loss: 0.9458857178688049
300/782 Loss: 1.0337421894073486
400/782 Loss: 0.9505584239959717
500/782 Loss: 0.8843657374382019
600/782 Loss: 0.9213059544563293
700/782 Loss: 1.0390369892120361
